<a href="https://colab.research.google.com/github/aaditya9803/ml/blob/main/Sorting_line_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# prompt: Write a code to open the csv files which is in the folders inside this with this id-  1BZJfMkmr5uyyL1iZGPQen-B3VTYrJjD0
# Just import these files and rename the files as (folder name)-(index of file), don't make dataframe..

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

folder_id = '1BZJfMkmr5uyyL1iZGPQen-B3VTYrJjD0'

# Get the list of folders inside the specified folder
folder_list = drive.ListFile({'q': "'{}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false".format(folder_id)}).GetList()


for folder in folder_list:
    folder_title = folder['title']
    print(f"Processing folder: {folder_title}")

    # Get the list of files inside the current folder
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder['id'])}).GetList()

    for index, file in enumerate(file_list):
        file_title = file['title']
        new_file_name = f"{folder_title}-{index}"
        print(f"Downloading {file_title} as {new_file_name}")

        # Download the file to the current runtime environment
        file.GetContentFile(new_file_name)

Processing folder: niO
Processing folder: biO
Processing folder: riO
Processing folder: wiO


In [135]:
import numpy as np
import pandas as pd
!pip install tsfresh
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.relevance import calculate_relevance_table



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.5/176.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 69.8 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.2 which is incompatible.


/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [133]:
def read_file(filename):
    return pd.read_csv((filename), sep=',')

index = index - 4
white = {}
white_df = pd.DataFrame()
blue = {}
blue_df = pd.DataFrame()
red = {}
red_df = pd.DataFrame()
nio = {}
nio_df = pd.DataFrame()

for j in range (index):
  white[j] = read_file(f"wiO-{j}")
  blue[j] = read_file(f"biO-{j}")
  red[j] = read_file(f"riO-{j}")
  nio[j] = read_file(f"niO-{j}")

In [117]:
len(white[j])

664

In [119]:
for j in range(index):
  white[j] = pd.DataFrame(white[j])
  white[j]['fileno'] = j
  white_df = pd.concat([white_df, white[j]])
  blue[j] = pd.DataFrame(blue[j])
  blue[j]['fileno'] = j
  blue_df = pd.concat([blue_df, blue[j]])
  red[j] = pd.DataFrame(red[j])
  red[j]['fileno'] = j
  red_df = pd.concat([red_df, red[j]])
  nio[j] = pd.DataFrame(nio[j])
  nio[j]['fileno'] = j
  nio_df = pd.concat([nio_df, nio[j]])

In [129]:
white_df ['color'] = 0
blue_df ['color'] = 1
red_df ['color'] = 2
nio_df ['color'] = 3

In [131]:
df = pd.concat([white_df, blue_df, red_df, nio_df])

In [141]:
df.head()

,Uhrzeit,I_In,V_In,P_In(W),I_Out(A),V_Out(V),P_Out(W),Temp(°C),Energie_In(Wh),Energie_Out(Wh),fileno,color
0,2024-05-02 11:37:37.940396:940,0.25,11.65,2.9125,0.29,8.97,2.6013,37.63,0.011648,0.009802,0,0
1,2024-05-02 11:37:37.948287:948,0.27,11.52,3.1104,0.25,9.00,2.2500,37.63,0.011654,0.009807,0,0
2,2024-05-02 11:37:37.956565:956,0.31,11.65,3.6115,0.27,9.00,2.4300,37.63,0.011663,0.009813,0,0
3,2024-05-02 11:37:37.968670:968,0.31,11.65,3.6115,0.25,9.03,2.2575,37.63,0.011675,0.009821,0,0
4,2024-05-02 11:37:37.977001:976,0.37,11.65,4.3105,0.33,9.03,2.9799,37.63,0.011685,0.009828,0,0


In [138]:
df = df.drop(columns=['Datum'])

In [143]:
features =  extract_features(
    df,
    column_id="fileno",
    column_sort="Uhrzeit",
    impute_function=impute
)

Feature Extraction:   3%|▎         | 26/880 [05:27<2:59:10, 12.59s/it]


KeyboardInterrupt: 